In [1]:
import pandas as pd 
import numpy as np
import rasterio as rio
import rasterio.features
from rasterio.warp import reproject
import geopandas as gpd
from shapely.wkt import loads
from shapely.geometry import Point
import matplotlib as mpl  
from matplotlib import pyplot as plt
from geocube.api.core import make_geocube
from osgeo import gdal
import scipy.ndimage as ndimage
from scipy.interpolate import griddata
from scipy.spatial import distance

In [22]:
edges = pd.read_csv('../network_inputs/bolinas_edges_sim.csv')
edges = gpd.GeoDataFrame(edges, crs='epsg:4326', geometry=edges['geometry'].map(loads))
edges['start_x'] = edges['geometry'].apply(lambda x: x.coords[0][0])
edges['end_x'] = edges['geometry'].apply(lambda x: x.coords[-1][0])

edges1 = edges.loc[edges['start_x']<=edges['end_x']]
cube = make_geocube(vector_data=edges1, measurements=["eid"], resolution=(-0.0002, 0.0002))
cube["eid"].rio.to_raster("../network_inputs/bolinas_edges_raster1.tif")

edges2 = edges.loc[edges['start_x']>edges['end_x']]
cube = make_geocube(vector_data=edges2, measurements=["eid"], resolution=(-0.0002, 0.0002))
cube["eid"].rio.to_raster("../network_inputs/bolinas_edges_raster2.tif")

In [4]:
### eucalyptus
# resample fire to match road

rst_fn = '../network_inputs/bolinas_edges_raster1.tif'
out_fn = 'fire/eucalyptus_match_roads.tif'

eucalyptus = pd.read_csv('fire/eucalyptus.csv')
eucalyptus = gpd.GeoDataFrame(eucalyptus, crs='epsg:4326', geometry=eucalyptus['WKT'].map(loads))
eucalyptus['value'] = 1

rst = rasterio.open(rst_fn)
meta = rst.meta.copy()
meta.update(compress='lzw')
with rasterio.open(out_fn, 'w+', **meta) as out:
    out_arr = out.read(1)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(eucalyptus.geometry, eucalyptus.value))

    burned = rasterio.features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)

# # open reference file and get resolution
# referenceFile = "../network_inputs/bolinas_edges_raster1.tif"
# reference = gdal.Open(referenceFile, 0)  # this opens the file in only reading mode
# referenceTrans = reference.GetGeoTransform()
# print(referenceTrans)
# x_min = referenceTrans[0]
# y_max = referenceTrans[3]
# x_res = referenceTrans[1]
# y_res = -referenceTrans[5]  # make sure this value is positive
# x_max = x_min + x_res*reference.RasterXSize
# y_min = y_max - y_res*reference.RasterYSize
# ref_proj = reference.GetProjection()
# print(x_min, y_min, x_max, y_max, x_res, y_res)

# inputFile = "fire/eucalyptus.tif"
# outputFile = "fire/eucalyptus_match_roads.tif"

# # ds = gdal.Warp(outputFile, inputFile)

# # dataset = gdal.Open(inputFile)
# # print(dataset.GetGeoTransform())
# # call gdal Warp
# kwargs = {"format": "GTiff", "xRes": x_res, "yRes": y_res, "dstSRS": ref_proj}#, "outputBounds": [ x_min, y_min, x_max, y_max]}
# ds = gdal.Warp(outputFile, inputFile, outputBounds=[ x_min, y_min, x_max, y_max], **kwargs)
# print(ds.GetGeoTransform())

# # band = ds.GetRasterBand(1)
# # print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

# # band_min = band.GetMinimum()
# # band_max = band.GetMaximum()
# # print(band_min, band_max)

In [4]:
template = rio.open("flamelength/flame_length_scen{}/{}.TIF".format(1, 1))

for fire_id in [3]:
    fire_time = template.read(1).copy()
    fire_time = np.nan
    flame_length = template.read(1).copy()
    flame_length = 0
    
    for hour_id in range(1, 11):
        fire_data = rio.open("flamelength/flame_length_scen{}/{}.TIF".format(fire_id, hour_id)).read(1)
        fire_time = np.where(np.isnan(fire_time) & (fire_data!=-9999), hour_id-1, fire_time)
        
#     first_hour_locations = np.mean(np.argwhere(fire_time==0), axis=0).astype(int)
#     fire_time[tuple(first_hour_locations)]=-1
#     print(first_hour_locations, fire_time[~np.isnan(fire_time)].shape)
    
#     positive_distance_list, negative_distance_list = [], []
#     distance_m = fire_time.copy()
#     distance_locations_m = np.argwhere(~np.isnan(distance_m))
#     print(fire_time.shape, distance_locations_m.shape, fire_time[~np.isnan(fire_time)].shape)
    
#     for hour_id in range(0, 10):
#         fire_time_locations_1 = np.argwhere(fire_time==(hour_id-1))
#         cdist_1 = np.min(distance.cdist(fire_time_locations_1, distance_locations_m), axis=0)
#         fire_time_locations_2 = np.argwhere(fire_time==hour_id)
#         cdist_2 = np.min(distance.cdist(fire_time_locations_2, distance_locations_m), axis=0)
#         positive_distance_list.append(cdist_1)
#         negative_distance_list.append(cdist_2)
#     positive_distance_array = np.vstack(positive_distance_list)
#     negative_distance_array = np.vstack(negative_distance_list)
#     positive_distance_array_expand = np.where(np.sum(positive_distance_array==0, axis=0)==0, 0, 1000)
#     positive_distance_array = np.vstack([positive_distance_array, positive_distance_array_expand])
#     display(positive_distance_array.shape)
 
#     before_id = np.argwhere(positive_distance_array.T==0)[:,1]-1
#     before_id = np.where(before_id<0, 0, before_id)
#     after_id = np.argwhere(positive_distance_array.T==0)[:,1]+1
#     after_id = np.where(after_id>9, 9, after_id)
#     before_distance = positive_distance_array[before_id, np.arange(positive_distance_array.shape[1]).tolist()]
#     after_distance = positive_distance_array[after_id, np.arange(positive_distance_array.shape[1]).tolist()]
#     time = before_id + before_distance/(before_distance+after_distance)
#     fire_time2 = fire_time.copy()
#     print(before_id.shape, after_id.shape, before_distance.shape, after_distance.shape, fire_time2[~np.isnan(fire_time2)].shape)
#     fire_time2[~np.isnan(fire_time2)] = time
#     print(fire_time2.shape)

    # output
#     with rasterio.open('flamelength/time_fire{}.tif'.format(fire_id), 'w', driver='GTiff', height=fire_time2.shape[0], width=fire_time2.shape[1], count=1, dtype=fire_time2.dtype, crs=template.crs, transform=template.transform) as dst:
#         dst.write(fire_time2, 1)

    # flame length
    flame_length = np.where(fire_data==-9999, np.nan, fire_data)
    print(np.max(flame_length[~np.isnan(flame_length)]))
    with rasterio.open('flamelength/flame_fire{}.tif'.format(fire_id), 'w', driver='GTiff', height=flame_length.shape[0], width=flame_length.shape[1], count=1, dtype=flame_length.dtype, crs=template.crs, transform=template.transform) as dst:
        dst.write(flame_length, 1)

13.645369


In [3]:
# resample fire to match road
# open reference file and get resolution
referenceFile = "../network_inputs/bolinas_edges_raster1.tif"
reference = gdal.Open(referenceFile, 0)  # this opens the file in only reading mode
referenceTrans = reference.GetGeoTransform()
print(referenceTrans)
x_min = referenceTrans[0]
y_max = referenceTrans[3]
x_res = referenceTrans[1]
y_res = -referenceTrans[5]  # make sure this value is positive
x_max = x_min + x_res*reference.RasterXSize
y_min = y_max - y_res*reference.RasterYSize
ref_proj = reference.GetProjection()
print(x_min, y_min, x_max, y_max, x_res, y_res)

for fire_id in [2, 3]:
    fire_time_inputFile = 'flamelength/time_fire{}.tif'.format(fire_id)
    flame_length_inputFile = 'flamelength/flame_fire{}.tif'.format(fire_id)
    for inputFile in [fire_time_inputFile, flame_length_inputFile]:
        outputFile = inputFile.split('.tif')[0] + '_match_road.tif'
        dataset = gdal.Open(inputFile)
        # call gdal Warp
        kwargs = {"format": "GTiff", "xRes": x_res, "yRes": y_res, "dstSRS": ref_proj}#, "outputBounds": [ x_min, y_min, x_max, y_max]}
        ds = gdal.Warp(outputFile, dataset, outputBounds=[ x_min, y_min, x_max, y_max], **kwargs)
        
        band = ds.GetRasterBand(1)
        print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

        min = band.GetMinimum()
        max = band.GetMaximum()
        if not min or not max:
            (min,max) = band.ComputeRasterMinMax(True)
        print("Min={:.3f}, Max={:.3f}".format(min,max))

        if band.GetOverviewCount() > 0:
            print("Band has {} overviews".format(band.GetOverviewCount()))

        if band.GetRasterColorTable():
            print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))

(-122.74640000000001, 0.00019999999999998825, 0.0, 37.9804, 0.0, -0.0001999999999999982)
-122.74640000000001 37.879200000000004 -122.59920000000001 37.9804 0.00019999999999998825 0.0001999999999999982
Band Type=Float64
Min=0.359, Max=9.500
Band Type=Float32
Min=0.110, Max=10.248
Band Type=Float64
Min=0.673, Max=9.500
Band Type=Float32
Min=0.116, Max=12.103


In [29]:
test = rio.open("flamelength/flame_fire3_match_road.tif")
test_array = test.read(1)
# np.max(test_array[~np.isnan(test_array)])
print(test_array.shape, np.max(test_array[~np.isnan(test_array)]))

(506, 736) 13.645369


In [19]:
inputFile = "flamelength/flame_length_scen{}/{}.TIF".format(1, 1)
dataset = gdal.Open(inputFile)

In [25]:
myarray = np.array(dataset.GetRasterBand(1).ReadAsArray())
myarray[myarray!=-9999]

array([ 0.57671535,  0.5642986 ,  0.548952  ,  0.62561786,  0.6142623 ,
        0.6094229 ,  0.58788407,  0.54788357,  2.1263912 ,  0.66354895,
        0.66777253,  0.675476  ,  0.67075735,  0.696022  ,  1.0820185 ,
        1.3953884 ,  0.6948105 ,  0.7186461 ,  0.74402833,  0.80314374,
        0.5722972 ,  0.8920534 ,  5.440207  ,  1.9779557 ,  0.5088464 ,
        0.75899535,  0.7954691 ,  0.860629  ,  1.1589137 ,  0.3401031 ,
        4.464514  ,  5.553088  ,  2.7849126 ,  0.4295089 ,  0.7955613 ,
        0.82697415,  0.8918342 ,  0.9390183 ,  0.7282005 ,  1.0255004 ,
        6.0939717 ,  2.2791307 ,  4.348677  ,  1.9713936 ,  1.2147585 ,
        0.85896915,  0.9205266 ,  0.8605279 ,  0.8025292 ,  0.88486797,
        1.1065257 ,  6.8664513 ,  3.0412977 ,  4.765931  ,  2.5612364 ,
        0.9408757 ,  0.8794874 ,  0.9394224 ,  0.95424795,  0.9064508 ,
        1.0177066 ,  1.1476529 ,  1.5042279 ,  5.673826  ,  3.6327436 ,
        4.394464  ,  5.0027747 ,  0.788056  ,  0.7338293 ,  0.99